In [1]:
import sys
import logging
import math

import glob
import os
import sys

import pandas as pd
import numpy as np
from dask.distributed import Client

from cesium import featurize

from sklearn.preprocessing import StandardScaler, MinMaxScaler, Normalizer

pd.options.display.max_rows = 1000
pd.options.display.max_columns = 1000

sys.path.insert(0, "/opt/vssexclude/personal/kaggle/k_tab_aug")

In [2]:
client = Client()

In [3]:
client.scheduler

<pooled rpc to 'tcp://127.0.0.1:43521'>

In [4]:
%load_ext autoreload
%autoreload 2
%config Completer.use_jedi = False

In [5]:
import src.config.constants as constants
import src.munging as process_data
import src.common as common

In [6]:
logger = common.get_logger("main")

In [9]:
features_to_use = ['all_times_nhist_numpeaks',
 'all_times_nhist_peak1_bin',
 'all_times_nhist_peak2_bin',
 'all_times_nhist_peak3_bin',
 'all_times_nhist_peak4_bin',
 'all_times_nhist_peak_1_to_2',
 'all_times_nhist_peak_1_to_3',
 'all_times_nhist_peak_1_to_4',
 'all_times_nhist_peak_2_to_3',
 'all_times_nhist_peak_2_to_4',
 'all_times_nhist_peak_3_to_4',
 'all_times_nhist_peak_val',
 'avg_double_to_single_step',
 'avg_err',
 'avgt',
 'cad_probs_1',
 'cad_probs_10',
 'cad_probs_20',
 'cad_probs_30',
 'cad_probs_40',
 'cad_probs_50',
 'cad_probs_100',
 'cad_probs_500',
 'cad_probs_1000',
 'cad_probs_5000',
 'cad_probs_10000',
 'cad_probs_50000',
 'cad_probs_100000',
 'cad_probs_500000',
 'cad_probs_1000000',
 'cad_probs_5000000',
 'cad_probs_10000000',
 'cads_avg',
 'cads_med',
 'cads_std',
 'mean',
 'med_double_to_single_step',
 'med_err',
 'n_epochs',
 'std_double_to_single_step',
 'std_err',
 'total_time',
 'amplitude',
 'flux_percentile_ratio_mid20',
 'flux_percentile_ratio_mid35',
 'flux_percentile_ratio_mid50',
 'flux_percentile_ratio_mid65',
 'flux_percentile_ratio_mid80',
 'max_slope',
 'maximum',
 'median',
 'median_absolute_deviation',
 'minimum',
 'percent_amplitude',
 'percent_beyond_1_std',
 'percent_close_to_median',
 'percent_difference_flux_percentile',
 'period_fast',
 'qso_log_chi2_qsonu',
 'qso_log_chi2nuNULL_chi2nu',
 'skew',
 'std',
 'stetson_j',
 'stetson_k',
 'weighted_average',
 'fold2P_slope_10percentile',
 'fold2P_slope_90percentile',
 'freq1_amplitude1',
 'freq1_amplitude2',
 'freq1_amplitude3',
 'freq1_amplitude4',
 'freq1_freq',
 'freq1_lambda',
 'freq1_rel_phase2',
 'freq1_rel_phase3',
 'freq1_rel_phase4',
 'freq1_signif',
 'freq2_amplitude1',
 'freq2_amplitude2',
 'freq2_amplitude3',
 'freq2_amplitude4',
 'freq2_freq',
 'freq2_rel_phase2',
 'freq2_rel_phase3',
 'freq2_rel_phase4',
 'freq3_amplitude1',
 'freq3_amplitude2',
 'freq3_amplitude3',
 'freq3_amplitude4',
 'freq3_freq',
 'freq3_rel_phase2',
 'freq3_rel_phase3',
 'freq3_rel_phase4',
 'freq_amplitude_ratio_21',
 'freq_amplitude_ratio_31',
 'freq_frequency_ratio_21',
 'freq_frequency_ratio_31',
 'freq_model_max_delta_mags',
 'freq_model_min_delta_mags',
 'freq_model_phi1_phi2',
 'freq_n_alias',
 'freq_signif_ratio_21',
 'freq_signif_ratio_31',
 'freq_varrat',
 'freq_y_offset',
 'linear_trend',
 'medperc90_2p_p',
 'p2p_scatter_2praw',
 'p2p_scatter_over_mad',
 'p2p_scatter_pfold_over_mad',
 'p2p_ssqr_diff_over_var',
 'scatter_res_raw']

In [11]:
train_df, test_df, combined_df = process_data.read_processed_data(logger=logger, data_dir=constants.PROCESSED_DATA_DIR)

[INFO]2021-08-27 13:10:26,282:main:Reading Data from /opt/vssexclude/personal/kaggle/k_tab_aug/data/processed...
[INFO]2021-08-27 13:10:26,601:main:Shape of train_df : (250000, 101)
[INFO]2021-08-27 13:10:26,743:main:Shape of test_df : (150000, 100)
[INFO]2021-08-27 13:10:26,754:main:Shape of sample_submission_df : (150000, 1)


In [12]:
combined_df = pd.concat([train_df.drop("loss", axis=1), test_df])

In [13]:
combined_df.head()

,f0,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,f12,f13,f14,f15,f16,f17,f18,f19,f20,f21,f22,f23,f24,f25,f26,f27,f28,f29,f30,f31,f32,f33,f34,f35,f36,f37,f38,f39,f40,f41,f42,f43,f44,f45,f46,f47,f48,f49,f50,f51,f52,f53,f54,f55,f56,f57,f58,f59,f60,f61,f62,f63,f64,f65,f66,f67,f68,f69,f70,f71,f72,f73,f74,f75,f76,f77,f78,f79,f80,f81,f82,f83,f84,f85,f86,f87,f88,f89,f90,f91,f92,f93,f94,f95,f96,f97,f98,f99
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,-0.002350,59,0.766739,-1.350460,42.272701,16.685699,30.359900,1.267300,0.392007,1.09101,1.96874,1.874640,117.286003,6.716200,0.985656,0.973428,4004232,0.377896,1.033820,0.574531,0.122371,8.18617,1517.829956,3.13219,0.112836,5.01868,116.765999,10891,8.193660,5.79715,1.09999,14.86840,-0.275434,0.915721,167.800003,-7.534140,4.23632,1.629560,1.144400,-0.303139,4.092090,3.22617,0.074799,0.259497,289.492004,327.464996,5.389310,7.39479,-0.489950,20.292299,2.45660,1.447720,-10639.000000,85.600502,-0.178513,2815,-234.772003,1.83317,88.560501,0.367916,8.575340e+09,70.973297,0.380057,0.031812,1.09527,0.563482,0.122689,1.160720,1.69391,1.07217,65.154297,0.022504,-5.606800,1.798660,0.528068,6696.299805,-0.562078,1.301020,6.71624,1.143470,2.29983,0.010485,-0.127223,0.231086,4.51614,0.594494,397,0.264022,8.68790,15.070100,0.376622,-42.439899,26.854000,1.45751,0.696161,0.941764,1.828470,0.924090,2.29658,10.48980
1,0.784462,145,-0.463845,-0.530421,27324.900391,3.475450,160.498001,0.828007,3.735860,1.28138,-2.73947,-0.529506,157.669998,0.696384,1.441230,0.159056,23567462,-0.089583,-0.711628,-1.045860,0.139920,3.69294,-123.353996,7.74097,-0.852302,8.51025,161.175003,87801,12.020200,1.78393,1.23100,10.14970,-0.018724,1.011280,127.401001,11.821400,5.99681,-0.953850,1.379370,1.079530,0.711923,2.90602,0.051206,0.755122,140.893005,29.525200,14.229600,3.53213,-0.405681,42.535702,1.43533,0.939771,138312.000000,59.881001,-0.070959,1435,1046.880005,1.56770,29.430599,2.455150,4.518200e+09,75.560204,1.987990,0.318177,1.14901,0.723574,0.184821,-0.348303,-7.17633,1.46258,43.112099,-0.060801,64.045502,2.358450,5.759690,3958.139893,1.576610,-1.241790,5.91412,0.959826,2.56631,0.000652,-0.335617,-0.271723,5.10319,8.706220,98,0.210513,7.86416,3.371900,0.147973,-184.132004,7.901370,1.70644,-0.494699,-2.058300,0.819184,0.439152,2.36470,1.14383
2,0.317816,19,-0.432571,-0.382644,1383.260010,19.712900,31.102600,-0.515354,34.430801,1.24210,2.90180,-0.960340,118.589996,7.696420,1.488760,0.387277,235760,0.055568,0.268560,0.718133,0.027133,11.67340,270.247009,3.44051,-0.679071,13.37810,150.362000,14173,1.699540,7.04728,1.10513,7.62871,0.685721,0.809485,120.064003,194.427002,6.77866,0.634136,0.934386,0.926980,0.741116,2.42205,0.260362,0.626639,369.579010,370.023987,4.488930,7.84290,1.196010,43.534302,2.59182,1.256710,168881.000000,83.841904,2.713230,2911,23256.900391,3.91896,97.557800,4.385380,8.444970e+08,99.493301,5.098040,0.258909,1.16878,0.049053,0.173547,0.937000,2.05947,1.22245,50.226700,-0.026215,71.684898,2.434170,1.904560,27165.800781,-0.773223,-1.833390,4.98548,1.170870,1.17201,0.016848,-0.235581,-0.724935,3.22565,4.170990,105,-0.155451,8.91829,0.186334,0.335985,7.437210,37.218102,3.25339,0.337934,0.615037,2.216760,0.745268,1.69679,12.30550
3,0.210753,17,-0.616454,0.946362,-119.252998,4.082350,185.257004,1.383310,-47.521400,1.09130,-1.51200,-1.292340,125.460999,7.343230,-3.092390,0.713795,1146032,0.326534,0.454842,0.219958,0.037982,5.15195,4893.859863,6.89751,-0.830552,4.43184,132.854996,77147,32.808998,4.06390,1.11911,3.91776,0.500353,0.206448,120.411003,233.537003,7.75131,0.625762,-1.940810,1.333210,-5.793170,1.56512,0.309483,0.134558,84.863701,24.335300,4.871200,2.55104,-0.372300,14.896000,1.10013,0.892193,17006.599609,78.407799,0.425686,1723,375.239990,1.94539,98.150002,-0.109164,3.543970e+09,60.808201,2.357590,0.239080,1.16116,0.535797,0.222185,1.163120,2.36343,1.41530,116.181999,0.018674,55.442799,2.228240,4.303600,2643.760010,-1.666320,0.792398,6.45162,1.077330,2.90676,

In [14]:
combined_df.shape

(400000, 100)

In [13]:
combined_df_min_max = combined_df.copy()
for name in combined_df.columns:
    mm = MinMaxScaler()
    combined_df_min_max.loc[:, name] = mm.fit_transform(combined_df[[name]])

In [14]:
combined_df_min_max = process_data.change_dtype(
        logger, combined_df_min_max, np.float64, np.float32
    )

[INFO]2021-08-27 10:22:36,232:main:Changing dtype of [f0] from [<class 'numpy.float64'>] to [<class 'numpy.float32'>]
[INFO]2021-08-27 10:22:36,295:main:Changing dtype of [f1] from [<class 'numpy.float64'>] to [<class 'numpy.float32'>]
[INFO]2021-08-27 10:22:36,355:main:Changing dtype of [f2] from [<class 'numpy.float64'>] to [<class 'numpy.float32'>]
[INFO]2021-08-27 10:22:36,412:main:Changing dtype of [f3] from [<class 'numpy.float64'>] to [<class 'numpy.float32'>]
[INFO]2021-08-27 10:22:36,468:main:Changing dtype of [f4] from [<class 'numpy.float64'>] to [<class 'numpy.float32'>]
[INFO]2021-08-27 10:22:36,524:main:Changing dtype of [f5] from [<class 'numpy.float64'>] to [<class 'numpy.float32'>]
[INFO]2021-08-27 10:22:36,581:main:Changing dtype of [f6] from [<class 'numpy.float64'>] to [<class 'numpy.float32'>]
[INFO]2021-08-27 10:22:36,637:main:Changing dtype of [f7] from [<class 'numpy.float64'>] to [<class 'numpy.float32'>]
[INFO]2021-08-27 10:22:36,692:main:Changing dtype of [f8

### Create inputs for cesium

In [15]:
a = []
for i in range(0, len(combined_df)):
    a.append(np.arange(start=0, stop=100, step=1))

In [21]:
a = []
for i in range(0, 100):
    a.append(np.arange(start=0, stop=100, step=1))

In [24]:
len(a), len(a[0])

(100, 100)

In [22]:
ts = list(combined_df_min_max[0:100].values)

In [23]:
len(ts), len(ts[0])

(100, 100)

In [30]:
from cesium import featurize
fset_cesium = featurize.featurize_time_series(times=a,
                                              values=ts,
                                              errors=None,
                                              features_to_use=features_to_use, scheduler="processes")

In [48]:
fset_cesium.columns = fset_cesium.columns.get_level_values(0)

In [51]:
fset_cesium.to_parquet("cesium.parquet", index=True)

In [15]:
df = pd.read_parquet(f"{constants.FEATURES_DATA_DIR}/cesium_features_1.parquet")

In [17]:
df.head()

feature,all_times_nhist_numpeaks,all_times_nhist_peak1_bin,all_times_nhist_peak2_bin,all_times_nhist_peak3_bin,all_times_nhist_peak4_bin,all_times_nhist_peak_1_to_2,all_times_nhist_peak_1_to_3,all_times_nhist_peak_1_to_4,all_times_nhist_peak_2_to_3,all_times_nhist_peak_2_to_4,all_times_nhist_peak_3_to_4,all_times_nhist_peak_val,avg_err,avgt,cad_probs_1,cad_probs_10,cad_probs_20,cad_probs_30,cad_probs_40,cad_probs_50,cad_probs_100,cad_probs_500,cad_probs_1000,cad_probs_5000,cad_probs_10000,cad_probs_50000,cad_probs_100000,cad_probs_500000,cad_probs_1000000,cad_probs_5000000,cad_probs_10000000,cads_avg,cads_med,cads_std,mean,med_err,n_epochs,std_err,total_time,amplitude,flux_percentile_ratio_mid20,flux_percentile_ratio_mid35,flux_percentile_ratio_mid50,flux_percentile_ratio_mid65,flux_percentile_ratio_mid80,max_slope,maximum,median,median_absolute_deviation,minimum,percent_amplitude,percent_beyond_1_std,percent_close_to_median,percent_difference_flux_percentile,period_fast,qso_log_chi2_qsonu,qso_log_chi2nuNULL_chi2nu,skew,std,stetson_j,stetson_k,weighted_average
0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.019896,0.0001,49.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.384039,0.0001,100.0,0.0,99.0,0.450840,0.191130,0.338361,0.452699,0.559850,0.772959,0.890374,0.924067,0.362063,0.149278,0.022386,0.404069,0.33,0.29,0.602679,3.305730,7.122306,-0.123582,0.563390,0.208602,1.085198,1.007635,0.384039
1,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.019896,0.0001,49.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.389141,0.0001,100.0,0.0,99.0,0.456057,0.143732,0.238170,0.437306,0.616274,0.794307,0.771707,0.952506,0.339848,0.146471,0.040393,0.431236,0.34,0.37,0.592125,11.713204,6.959596,0.051253,0.562886,0.209870,0.997002,0.986508,0.389141
2,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.019896,0.0001,49.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.414719,0.0001,100.0,0.0,99.0,0.413268,0.203387,0.361056,0.544165,0.681001,0.904587,0.648589,0.867214,0.415744,0.160334,0.040679,0.412608,0.38,0.28,0.562971,65.303430,6.766104,0.152233,0.037125,0.200383,1.097396,1.058024,0.414719
3,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.019896,0.0001,49.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.370996,0.0001,100.0,0.0,99.0,0.392384,0.226197,0.321468,0.458245,0.606283,0.787765,0.733048,0.841225,0.326606,0.132212,0.056456,0.377504,0.29,0.28,0.534470,4.935687,6.795354,0.067326,0.603137,0.194884,0.904983,1.010470,0.370996
4,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.019896,0.0001,49.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.376182,0.0001,100.0,0.0,99.0,0.400262,0.177094,0.319867,0.476586,0.571985,0.773294,0.729045,0.824837,0.381530,0.146866,0.024313,0.389590,0.33,0.30,0.582511,29.220779,6.697120,0.118499,0.204622,0.190352,0.932655,1.035006,0.376182
